In [20]:
from google.cloud import bigquery
import pandas as pd

In [10]:
client = bigquery.Client()

In [24]:
sql = """
SELECT
      CONCAT(
        'https://stackoverflow.com/questions/',
        CAST(id as STRING)) as url,
      view_count
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE tags like '%google-bigquery%'
    ORDER BY view_count DESC
    LIMIT 10
"""

In [26]:
pd.read_gbq(sql, dialect = "standard")

,url,view_count
0,https://stackoverflow.com/questions/22879669,52306
1,https://stackoverflow.com/questions/13530967,46073
2,https://stackoverflow.com/questions/35159967,45991
3,https://stackoverflow.com/questions/10604135,45238
4,https://stackoverflow.com/questions/16609219,37758
5,https://stackoverflow.com/questions/11647201,32963
6,https://stackoverflow.com/questions/13221978,32507
7,https://stackoverflow.com/questions/27060396,31630
8,https://stackoverflow.com/questions/6607552,31487
9,https://stackoverflow.com/questions/11057219,29069
